CAN I SAVE THE CHANGES???

In [4]:
!pip install uniparser_morph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/DionysiusSidorius/project-753.git

Cloning into 'project-753'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 62 (delta 14), reused 31 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 798.47 KiB | 18.15 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [1]:
with open('lexemes.txt', 'w') as lex:
  lex.write(
      """-lexeme
 lex: test
 stem: test.
 gramm: NOUN
 paradigm: NOM
 trans_en: testus
      """
  )
with open('paradigms.txt', 'w') as par:
  par.write(
      """-paradigm: NOM
 -flex: .
  gramm: sg
 -flex: .s
  gramm: pl
  gloss: PL
 -flex: .'s
  gramm: sg,poss
  gloss: POSS
 -flex: .s'
  gramm: pl,poss
  gloss: POSS.PL
      """
  )


In [5]:
from uniparser_morph import Analyzer
a = Analyzer()
a.load_grammar()

In [6]:
analyses = a.analyze_words(['tests', "test's"])
print(analyses)

[[<Wordform object>
tests
test; NOUN,pl
test-s
STEM-PL
trans_en	testus
], [<Wordform object>
test's
test; NOUN,poss,sg
test-'s
STEM-POSS
trans_en	testus
]]
